## Deezer playlist to csv

In [ ]:
import requests
import pandas as pd

playlist_id_list = ["13301519863",
                    "11109924384",
                    "11109924124",
                    "11109918904",
                    "11109919544",
                    "11917580341",
                    "11109924924",
                    "11109923824",
                    "11912452141",
                    "11109920664",
                    "11537618704",
                    "11164434904",
                    "12126532871",
                    "11109921284",
                    "11917626441",
                    "11959053381",
                    "11109921064",
                    ]

df_tracks = pd.DataFrame(columns=["id", "title", "album", "duration", "release_date", "rank", "bpm", "gain"])

for playlist_id in playlist_id_list:
    response = requests.get(f"https://api.deezer.com/playlist/{playlist_id}")
    playlist = response.json()
    playlist_name = f'in_{playlist["title"].replace(" ", "_")}'

    track_id_list = [playlist["tracks"]["data"][i]["id"] for i in range(len(playlist["tracks"]["data"]))]

    for track_id in track_id_list:
        if track_id not in df_tracks["id"].values:
            response = requests.get(f"https://api.deezer.com/track/{track_id}")
            track = response.json()
            artists = ([artist["name"] for artist in track["contributors"]])
            df_tracks = pd.concat([df_tracks, pd.DataFrame([{
                "id": track["id"],
                "title": track["title"],
                "album": track["album"]["title"],
                "duration": track["duration"],
                "release_date": track["album"]["release_date"],
                "rank": track["rank"],
                "bpm": track["bpm"],
                "gain": track["gain"],
                f"{playlist_name}": True
            }])], ignore_index=True)

            for i in range(len(artists)):
                df_tracks.loc[df_tracks.index[-1], f'artist_{i+1}'] = artists[i] if i < len(artists) else None

        else:
            df_tracks.loc[df_tracks["id"] == track_id, f'{playlist_name}'] = True


artist_columns = [col for col in df_tracks.columns if col.startswith("artist")]
in_columns = [col for col in df_tracks.columns if col.startswith("in_")]
other_columns = [col for col in df_tracks.columns if col not in artist_columns and col not in in_columns]
df_tracks = df_tracks[other_columns + artist_columns + in_columns]
       
df_tracks.head()

In [ ]:
duplicate_titles = df_tracks[df_tracks.duplicated(subset='title', keep=False)]
duplicate_titles

In [ ]:
=((B2*0,1)*1+(C2*0,285))

In [ ]:
all_artists = df_tracks[artist_columns].stack().reset_index(level=1, drop=True).to_frame('artist')
all_artists['in_fav'] = df_tracks.loc[all_artists.index, 'in_fav'].values

artist_counts = all_artists['artist'].value_counts().to_frame('nombre_de_titres')
fav_counts = all_artists[all_artists['in_fav'] == True]['artist'].value_counts().to_frame('nombre_de_favoris')

artist_summary = artist_counts.join(fav_counts, how='left').fillna(0)
artist_summary['coef'] = round((artist_summary['nombre_de_titres'] * 0.1) * (1 + artist_summary['nombre_de_favoris'] * 0.25), 3)
artist_summary.reset_index(inplace=True)
artist_summary.rename(columns={'index': 'artiste'}, inplace=True)

artist_summary